# data fetching 
### via `psycopg` or `sqlalchemy`

### 1.) `psycopg`

### imports

In [1]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

### database connection

In [2]:
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

### cursor

In [3]:
cur = conn.cursor()

### run query and fetch data

In [4]:
cur.execute('SELECT * FROM eda.king_county_house_sales LIMIT 10')
cur.fetchall()

[(datetime.date(2014, 10, 13), 221900.0, 7129300520, 1),
 (datetime.date(2014, 12, 9), 538000.0, 6414100192, 2),
 (datetime.date(2015, 2, 25), 180000.0, 5631500400, 3),
 (datetime.date(2014, 12, 9), 604000.0, 2487200875, 4),
 (datetime.date(2015, 2, 18), 510000.0, 1954400510, 5),
 (datetime.date(2014, 5, 12), 1230000.0, 7237550310, 6),
 (datetime.date(2014, 6, 27), 257500.0, 1321400060, 7),
 (datetime.date(2015, 1, 15), 291850.0, 2008000270, 8),
 (datetime.date(2015, 4, 15), 229500.0, 2414600126, 9),
 (datetime.date(2015, 3, 12), 323000.0, 3793500160, 10)]

### import into pandas dataframe

In [5]:
query_string = "SELECT * FROM eda.king_county_house_sales LIMIT 10"
df_psycopg = pd.read_sql(query_string, conn)

/var/folders/78/tz8wl15n2fnc6zfgmbfjzhwm0000gn/T/ipykernel_6492/3156748183.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_psycopg = pd.read_sql(query_string, conn)


### close connection

In [6]:
conn.close()

### show head of dataframe

In [7]:
df_psycopg.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


### export to csv

In [8]:
df_psycopg.to_csv('data/psycopg.csv',index=False)

### 2.) `sqlalchemy`

### imports

In [9]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

#read the database string from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

### engine to connect

In [10]:
db = create_engine(DB_STRING)

### import everything into pandas dataframe

In [11]:
query_string = "SELECT * FROM eda.king_county_house_sales"
df_sqlalchemy = pd.read_sql(query_string, db)

### show head of dataframe

In [12]:
df_sqlalchemy.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


### export to csv

In [13]:
df_sqlalchemy.to_csv('data/sqlalchemy.csv',index=False)

# data table merging

### import csv to dataframe

In [14]:
df_sales = pd.read_csv('data/sqlalchemy.csv')

# setting 'id' column as the index
df_sales.set_index('id', inplace=True)
df_sales

,date,price,house_id
id,,,
1,2014-10-13,221900.0,7129300520
2,2014-12-09,538000.0,6414100192
3,2015-02-25,180000.0,5631500400
4,2014-12-09,604000.0,2487200875
5,2015-02-18,510000.0,1954400510
...,...,...,...
21593,2014-05-21,360000.0,263000018
21594,2015-02-23,400000.0,6600060120
21595,2014-06-23,402101.0,1523300141


### basic eda of raw sales data

In [15]:
df_sales.describe()

,price,house_id
count,2.159700e+04,2.159700e+04
mean,5.402966e+05,4.580474e+09
std,3.673681e+05,2.876736e+09
min,7.800000e+04,1.000102e+06
25%,3.220000e+05,2.123049e+09
50%,4.500000e+05,3.904930e+09
75%,6.450000e+05,7.308900e+09
max,7.700000e+06,9.900000e+09


In [16]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21597 entries, 1 to 21597
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      21597 non-null  object 
 1   price     21597 non-null  float64
 2   house_id  21597 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 674.9+ KB


### fetching data for home details -- to combine with sales

In [17]:
query_string = "SELECT * FROM eda.king_county_house_details"
df_details = pd.read_sql(query_string, db)
df_details.set_index('id', inplace=True)
df_details

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,
1000102,6.0,3.00,2400.0,9373.0,2.0,NaN,0.0,3,7,2400.0,0.0,1991,0.0,98002,47.3262,-122.214,2060.0,7316.0
100100050,3.0,1.00,1320.0,11090.0,1.0,0.0,0.0,3,7,1320.0,0.0,1955,0.0,98155,47.7748,-122.304,1320.0,8319.0
1001200035,3.0,1.00,1350.0,7973.0,1.5,NaN,0.0,3,7,1350.0,0.0,1954,0.0,98188,47.4323,-122.292,1310.0,7491.0
1001200050,4.0,1.50,1260.0,7248.0,1.5,NaN,0.0,5,7,1260.0,0.0,1955,NaN,98188,47.4330,-122.292,1300.0,7732.0
1003000175,3.0,1.00,980.0,7606.0,1.0,0.0,0.0,3,7,980.0,0.0,1954,0.0,98188,47.4356,-122.290,980.0,8125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993002177,3.0,2.50,1380.0,1547.0,3.0,0.0,0.0,3,8,1380.0,0.0,2000,NaN,98103,47.6908,-122.341,1380.0,1465.0
993002225,3.0,2.25,1520.0,1245.0,3.0,NaN,0.0,3,8,1520.0,0.0,2004,0.0,98103,47.6907,-122.340,1520.0,1470.0
993002247,3.0,2.25,1550.0,1469.0,3.0,0.0,0.0,3,8,1550.0,0.0,2004,0.0,98103,47.6911,-122.341,1520.0,1465.0


### basic eda of raw details data

In [18]:
df_details.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21420.000000,21420.000000,21420.000000,2.142000e+04,21420.000000,19060.000000,21357.000000,21420.000000,21420.000000,21420.000000,20969.000000,21420.000000,17609.000000,21420.00000,21420.000000,21420.000000,21420.000000,21420.000000
mean,3.373950,2.118429,2083.132633,1.512804e+04,1.495985,0.007660,0.234677,3.410784,7.662792,1791.170215,292.086938,1971.092997,838.805724,98077.87437,47.560197,-122.213784,1988.384080,12775.718161
std,0.925405,0.768720,918.808412,4.153080e+04,0.540081,0.087188,0.766459,0.650035,1.171971,828.692965,442.695597,29.387141,4005.127226,53.47748,0.138589,0.140791,685.537057,27345.621867
min,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.00000,47.155900,-122.519000,399.000000,651.000000
25%,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,98033.00000,47.471200,-122.328000,1490.000000,5100.000000
50%,3.000000,2.250000,1920.000000,7.614000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.00000,47.572100,-122.230000,1840.000000,7620.000000
75%,4.000000,2.500000,2550.000000,1.069050e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2220.000000,560.000000,1997.000000,0.000000,98117.00000,47.678100,-122.125000,2370.000000,10086.250000
max,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,20150.000000,98199.00000,47.777600,-121.315000,6210.000000,871200.000000


In [19]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21420 entries, 1000102 to 999000215
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       21420 non-null  float64
 1   bathrooms      21420 non-null  float64
 2   sqft_living    21420 non-null  float64
 3   sqft_lot       21420 non-null  float64
 4   floors         21420 non-null  float64
 5   waterfront     19060 non-null  float64
 6   view           21357 non-null  float64
 7   condition      21420 non-null  int64  
 8   grade          21420 non-null  int64  
 9   sqft_above     21420 non-null  float64
 10  sqft_basement  20969 non-null  float64
 11  yr_built       21420 non-null  int64  
 12  yr_renovated   17609 non-null  float64
 13  zipcode        21420 non-null  int64  
 14  lat            21420 non-null  float64
 15  long           21420 non-null  float64
 16  sqft_living15  21420 non-null  float64
 17  sqft_lot15     21420 non-null  float64
dtypes

### comparing dataframe shapes

In [20]:
df_details.shape

(21420, 18)

In [21]:
df_sales.shape

(21597, 3)

### joining details table to sales table, pairing 'house_id' and 'id'

In [22]:
df_combined = pd.merge(df_sales, df_details, left_on='house_id', right_index=True, how='left')
df_combined.set_index(df_sales.index, inplace=True)
df_combined

,date,price,house_id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,,
1,2014-10-13,221900.0,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,...,7,1180.0,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0
2,2014-12-09,538000.0,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,...,7,2170.0,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0
3,2015-02-25,180000.0,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,...,6,770.0,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0
4,2014-12-09,604000.0,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,...,7,1050.0,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0
5,2015-02-18,510000.0,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,...,8,1680.0,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21593,2014-05-21,360000.0,263000018,3.0,2.50,1530.0,1131.0,3.0,0.0,0.0,...,8,1530.0,0.0,2009,0.0,98103,47.6993,-122.346,1530.0,1509.0
21594,2015-02-23,400000.0,6600060120,4.0,2.50,2310.0,5813.0,2.0,0.0,0.0,...,8,2310.0,0.0,2014,0.0,98146,47.5107,-122.362,1830.0,7200.0
21595,2014-06-23,402101.0,1523300141,2.0,0.75,1020.0,1350.0,2.0,0.0,0.0,...,7,1020.0,0.0,2009,0.0,98144,47.5944,-122.299,1020.0,2007.0


### basic eda of combined table

In [23]:
df_combined.shape

(21597, 21)

In [24]:
df_combined.columns

Index(['date', 'price', 'house_id', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [25]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21597 entries, 1 to 21597
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21597 non-null  object 
 1   price          21597 non-null  float64
 2   house_id       21597 non-null  int64  
 3   bedrooms       21597 non-null  float64
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  float64
 6   sqft_lot       21597 non-null  float64
 7   floors         21597 non-null  float64
 8   waterfront     19206 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  float64
 13  sqft_basement  21145 non-null  float64
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17749 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long       

# data cleaning

### import numpy

In [26]:
import numpy as np

### lower_snake_case columns

In [27]:
df_combined.columns = df_combined.columns.str.replace(' ','_').str.lower()
df_combined.columns

Index(['date', 'price', 'house_id', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

### no value count duplicates

In [28]:
df_combined.duplicated().value_counts()

False    21597
Name: count, dtype: int64

### datatype by column

In [29]:
df_combined.dtypes

date              object
price            float64
house_id           int64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot         float64
floors           float64
waterfront       float64
view             float64
condition          int64
grade              int64
sqft_above       float64
sqft_basement    float64
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
dtype: object

### date datatype to datetime instead of object

In [30]:
df_combined['date'] = pd.to_datetime(df_combined['date'], format='%Y-%m-%d')

In [31]:
df_combined.dtypes

date             datetime64[ns]
price                   float64
house_id                  int64
bedrooms                float64
bathrooms               float64
sqft_living             float64
sqft_lot                float64
floors                  float64
waterfront              float64
view                    float64
condition                 int64
grade                     int64
sqft_above              float64
sqft_basement           float64
yr_built                  int64
yr_renovated            float64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15           float64
sqft_lot15              float64
dtype: object

In [32]:
df_combined['yr_built'].unique()

array([1955, 1951, 1933, 1965, 1987, 2001, 1995, 1963, 1960, 2003, 1942,
       1927, 1977, 1900, 1979, 1994, 1916, 1921, 1969, 1947, 1968, 1985,
       1941, 1915, 1909, 1948, 2005, 1929, 1981, 1930, 1904, 1996, 2000,
       1984, 2014, 1922, 1959, 1966, 1953, 1950, 2008, 1991, 1954, 1973,
       1925, 1989, 1972, 1986, 1956, 2002, 1992, 1964, 1952, 1961, 2006,
       1988, 1962, 1939, 1946, 1967, 1975, 1980, 1910, 1983, 1978, 1905,
       1971, 2010, 1945, 1924, 1990, 1914, 1926, 2004, 1923, 2007, 1976,
       1949, 1999, 1901, 1993, 1920, 1997, 1943, 1957, 1940, 1918, 1928,
       1974, 1911, 1936, 1937, 1982, 1908, 1931, 1998, 1913, 2013, 1907,
       1958, 2012, 1912, 2011, 1917, 1932, 1944, 1902, 2009, 1903, 1970,
       2015, 1934, 1938, 1919, 1906, 1935])

### something is odd with year renovated as strange floats

In [33]:
df_combined['yr_renovated'].unique()

array([    0., 19910.,    nan, 20020., 20100., 19920., 20130., 19940.,
       19780., 20050., 20030., 19840., 19540., 20140., 20110., 19830.,
       19450., 19900., 19880., 19770., 19810., 19950., 20000., 19990.,
       19980., 19700., 19890., 20040., 19860., 20070., 19870., 20060.,
       19850., 20010., 19800., 19710., 19790., 19970., 19500., 19690.,
       19480., 20090., 20150., 19740., 20080., 19680., 20120., 19630.,
       19510., 19620., 19530., 19930., 19960., 19550., 19820., 19560.,
       19400., 19760., 19460., 19750., 19640., 19730., 19570., 19590.,
       19600., 19670., 19650., 19340., 19720., 19440., 19580.])

In [34]:
df_combined['yr_renovated'] = df_combined.yr_renovated.replace(0.0,np.NaN)
df_combined['yr_renovated']

id
1            NaN
2        19910.0
3            NaN
4            NaN
5            NaN
          ...   
21593        NaN
21594        NaN
21595        NaN
21596        NaN
21597        NaN
Name: yr_renovated, Length: 21597, dtype: float64

### fixing the year renovated column to either a year or Not a Time 

In [35]:
df_combined['yr_renovated'] = df_combined['yr_renovated'].apply(lambda x: pd.to_datetime(int(x / 10), format='%Y').year if pd.notna(x) else pd.NaT)
df_combined['yr_renovated']

id
1         NaT
2        1991
3         NaT
4         NaT
5         NaT
         ... 
21593     NaT
21594     NaT
21595     NaT
21596     NaT
21597     NaT
Name: yr_renovated, Length: 21597, dtype: object

### checking houses with multiple sales

In [36]:
duplicate_house_ids = df_combined[df_combined.duplicated(subset=['house_id'], keep=False)]
duplicate_house_ids

,date,price,house_id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,,
94,2014-07-25,430000.0,6021501535,3.0,1.50,1580.0,5000.0,1.0,0.0,0.0,...,8,1290.0,290.0,1939,NaT,98117,47.6870,-122.386,1570.0,4500.0
95,2014-12-23,700000.0,6021501535,3.0,1.50,1580.0,5000.0,1.0,0.0,0.0,...,8,1290.0,290.0,1939,NaT,98117,47.6870,-122.386,1570.0,4500.0
314,2014-06-18,1380000.0,4139480200,4.0,3.25,4290.0,12103.0,1.0,0.0,3.0,...,11,2690.0,1600.0,1997,NaT,98006,47.5503,-122.102,3860.0,11244.0
315,2014-12-09,1400000.0,4139480200,4.0,3.25,4290.0,12103.0,1.0,0.0,3.0,...,11,2690.0,1600.0,1997,NaT,98006,47.5503,-122.102,3860.0,11244.0
325,2014-09-05,232000.0,7520000520,2.0,1.00,1240.0,12092.0,1.0,0.0,0.0,...,6,960.0,280.0,1922,1984,98146,47.4957,-122.352,1820.0,7460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20655,2015-03-30,502000.0,8564860270,4.0,2.50,2680.0,5539.0,2.0,NaN,0.0,...,8,2680.0,0.0,2013,NaN,98045,47.4759,-121.734,2680.0,5992.0
20764,2014-06-26,240000.0,6300000226,4.0,1.00,1200.0,2171.0,1.5,0.0,0.0,...,7,1200.0,0.0,1933,NaN,98133,47.7076,-122.342,1130.0,1598.0
20765,2015-05-04,380000.0,6300000226,4.0,1.00,1200.0,2171.0,1.5,0.0,0.0,...,7,1200.0,0.0,1933,NaN,98133,47.7076,-122.342,1130.0,1598.0


### creating price per square foot

In [37]:
df_combined['price_sqft'] = df_combined['price'] / df_combined['sqft_living']
df_combined['price_sqft']

id
1        188.050847
2        209.338521
3        233.766234
4        308.163265
5        303.571429
            ...    
21593    235.294118
21594    173.160173
21595    394.216667
21596    250.000000
21597    318.627451
Name: price_sqft, Length: 21597, dtype: float64

# final cleaned export to csv file 

In [38]:
df_combined.to_csv('data/everything.csv',index=True)